Runge Kuta w/ adaptive step-size control 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 

def dfdx(x,f):
    return x**2 + x

def f_int(x,C):
    return (x**3)/3. + 0.5*x**2 + C

def rk2_core(x_i,f_i,h,g):
    
    #applying adaptive step-size control
    
    f_ipo = f_i + h*g(x_i + (1/5)*h, f_i + (1/5)*h*g(x_i,f_i))
    
    return f_ipo

def rk2(dfdx,a,b,f_a,N):
    
    #dfdx is the derivative wrt to x
    #a is the lower bound
    #b is the upper bound
    #f_a is the boundary condition at a 
    #N is the number of steps 
    
    #define our steps
    x = np.linspace(a,b,N)
    
    #a single step size
    h = x[1]-x[0]
    
    #an array to hold f 
    f = np.zeros(N,dtype=float)
    
    f[0] = f_a #values of f at a 
    
    #evolve f along x 
    for i in range(1,N):
        f[i] = rk2_core(x[i-1],f[i-1],h,dfdx)
        
    return x,f

def rk4_core(x_i,f_i,h,g):
    
    #applying adaptive step-size control for k of 1 through 4 
    
    k_1 = h*g(x_i,f_i)
    k_2 = h*g(x_i + (1/5)*h,f_i + (1/5)*k_1)
    k_3 = h*g(x_i + (3/10)*h,f_i + (3/40)*k_1 + (9/40)*k_2)
    k_4 = h*g(x_i + (3/5)*h,f_i + (3/10)*k_1 - (9/10)*k_2 + (6/5)*k_3)
    
    f_ipo = f_i + (k_1 + 2*k_2 + 2*k_3 + k_4)/6.
    
    return f_ipo 

def rk4(dfdx,a,b,f_a,N):
    
    #dfdx is the derivative wrt to x
    #a is the lower bound
    #b is the upper bound
    #f_a is the boundary condition at a 
    #N is the number of steps 
    
    #define our steps
    x = np.linspace(a,b,N)
    
    #a single step size 
    h = x[1]-x[0]
    
    #an array to hold f 
    f = np. zeros(N, dtype=float)
    
    f[0] = f_a #values of f at a
    
    #evolve f along x
    for i in range(1,N):
        f[i] = rk4_core(x[i-1],f[i-1],h,dfdx)
    
    return x,f

a = 0.0
b = 1.0 
f_a = 0.0
N = 20 #change to 20 for better fitting
x_2, f_2 = rk2(dfdx,a,b,f_a,N)
x_4, f_4 = rk4(dfdx,a,b,f_a,N)
x = x_2.copy()
plt.plot(x_2,f_2,label="RK2")
plt.plot(x_4,f_4,label="RK4")
plt.plot(x,f_int(x,f_a),'o',label="Analytic")
plt.legend(frameon=False)

Coupled ODE

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def dydx(x,y):
    #set the derivatives
    #our equation is d^2y/dx^2 = -y
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we well ser z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs

def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? array
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array 
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values 
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + 0.5*k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        #estimate our error by taking one step of size h 
        #vs. two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #compute an error 
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, take a smaller step 
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("Too many iteration in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        
        #iterate 
        i+=1
    
    #next time, try to take a bigger step
    h_new = np. fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step was actually took 
    return y_2, h_new, h_step 

def rk4_mv(dfdx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary condition
    #tol is the tolerance for integrating y
    
    #define our starting step 
    xi = a
    yi = y_a.copy()
    
    #an initial step size  == make very small !!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iteration
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the 
    #size of y_a
    nv = len (y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1 
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new 
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b - xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break 
            flag = 0 
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new 
        
        #prevent too many iterations
        if(i>imax):
            
            print("Maximum iterations has recahed.")
            raise StopIteration("Iteration number = ", i)
            
        #iterate
        i+=1
        
        #output some information
        s = "i = %3d\tx = %9.8f/th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0 
            
    return x,y 

a = 0.0 
b= 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2 

tolerance = 1.0e-6

#perform the integration 
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

#Plotting the values

plt.figure()
plt.plot(x,y[:,0],'o',label="y(x)")
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi, 1000)
plt.plot(xx,np.sin(xx),label = "sin(x)")
plt.plot(xx,np.cos(xx),label = "cos(x)")
plt.xlabel('x')
plt.ylabel('y,dy/dx')
plt.xlim([0,2*np.pi])
plt.legend(frameon=False)

#Plotting the error

plt.figure()
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error, label="y(x) Error")
plt.plot(x,dydx_error,label="dydx(x) Error")
plt.xlim([0,2*np.pi])
plt.legend(frameon=False)